In [2]:
%matplotlib nbagg
import numpy as np
import json
import os
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

from alvra_tools import load_FEL_scans
from alvra_tools import errfunc
from alvra_tools.channels_res import *

In [3]:
scan_name ="knife_edge-x_002"

DIR = "/sf/alvra/data/p17805/res/scan_data/" + scan_name + "/"
DIR_json = "/sf/alvra/data/p17805/res/scan_info/"

In [11]:
!ls -lah -rt $DIR
!hostname

listfile = os.listdir(DIR)
number_files = len(listfile)
print ("There are",number_files,"files in the folder", DIR)

total 141M
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:18 knife_edge-x_002_step0000.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:18 knife_edge-x_002_step0001.h5
drwxr-sr-x. 4 39147 p17805 4.0K May 20 16:18 ..
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:18 knife_edge-x_002_step0002.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:18 knife_edge-x_002_step0003.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:18 knife_edge-x_002_step0004.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:18 knife_edge-x_002_step0005.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:18 knife_edge-x_002_step0006.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:19 knife_edge-x_002_step0007.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:19 knife_edge-x_002_step0008.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:19 knife_edge-x_002_step0009.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:19 knife_edge-x_002_step0010.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:19 knife_edge-x_002_step0011.h5
-rw-r--r--. 1 39147 p17805 5.6M May 20 16:19 knife_ed

In [27]:
json_file = DIR_json + scan_name + "_scan_info.json"
print (json_file)

with open(json_file) as file:
    data = json.load(file)
numFiles = len(data['scan_files'])
#numFiles = 1

print ("Processing",numFiles,"files")

Signal = []
err_Signal = []

Izero = []
err_Izero = []

Position_mm = []

for i in range(0,numFiles):
    filename = str(data['scan_files'][i][0])
    exists = os.path.isfile(filename)
    if exists:
        print("step",i+1,"of",numFiles,": Processing %s" % (str(data['scan_files'][i][0])))
        (data, Izero, position, pulseIDs) = load_FEL_scans_pulseID(filename, channel_position_X, 25)
        Signal.append(data.mean())
        err_Signal.append(data.std()/np.sqrt(data.size))
        
  #      Izero.append(Izero.mean())
  #      err_Izero.append(Izero.std()/np.sqrt(Izero.size))
    
  #      position = [x for x in position if (np.abs(x) > 0)]
        Position_mm.append(np.mean(position))
    #Position_mm.append(position_loop[0])
    
Signal = np.asarray(Signal)
err_Signal = np.asarray(err_Signal)

Izero = np.asarray(Izero)
err_Izero = np.asarray(err_Izero)

Position_mm = np.asarray(Position_mm)
print ("Job done!")

/sf/alvra/data/p17805/res/scan_info/knife_edge-x_002_scan_info.json
Processing 26 files
step 1 of 26 : Processing /sf/alvra/data/p17805/res/scan_data/knife_edge-x_002/knife_edge-x_002_step0000.h5


IndexError: invalid index to scalar variable.

In [ ]:
#Normalization

Signal_norm = Signal / Izero
err_signal_norm = err_signal / Izero

In [ ]:
plt.figure()

plt.plot(Position_mm, Signal_norm, label="FEL",marker='.')
low_err= Signal_norm - err_signal_norm
high_err= Signal_norm + err_signal_norm


plt.fill_between(Position_mm, low_err, high_err, color='lightblue')
plt.title(DIR)
plt.xlabel("Position (mm)")
plt.legend(loc="best")


plt.show()
#plt.savefig('Timescan_7_diode')

In [ ]:
# Now fit with error function (defined above)
parameters,extras = curve_fit(errfunc, Position_mm, Signal_norm, p0 = [0, 50000, 12.1, 0.5]) # [baseline, step amplitude, center, FWHM]

print("Position t0 =",(np.round(parameters[2],3)),"mm")
print("Width =",np.abs(np.round(parameters[3],3)),"mm")

In [ ]:
#... and the plot with the fit curve

#savedir = "/das/work/p17/p17569/Beamline/"

plt.figure()

plt.plot(Position_mm, Signal_norm, label="label here",marker='.')
plt.plot(Position_mm, errfunc(Position_mm,*parameters))

low_err= Signal_norm - err_signal_norm
high_err= Signal_norm + err_signal_norm

plt.fill_between(Position_mm, low_err, high_err, color='lightblue')

plt.title(np.round(np.abs(parameters[3])*1000, 4) )
#plt.title("X direction")
plt.grid()
plt.xlabel("Position (mm)")
plt.legend(loc="lower left")
plt.show()
#plt.savefig(savedir + 'scan_LAM_003')